In [1]:
import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display
from torch import nn

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

C:\Users\user\.virtualenvs\xlab-ml-service\Lib\site-packages\torch\_subclasses\functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


# Loading silero v1 TTS model

In [63]:
import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display
from torch import nn

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

language = 'ru'
speaker = 'kseniya_16khz'
device = torch.device('cuda:0')
model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                      model='silero_tts',
                                                                      language=language,
                                                                      speaker=speaker)
model = model.to(device)

Using cache found in C:\Users\user/.cache\torch\hub\snakers4_silero-models_master


## View the model layers

### Model tacotron block

In [ ]:
torch.jit.script(model.tacotron).code

In [94]:
from typing import Optional, Tuple
from torch import Tensor


def forward(self,
            inputs: Tensor,
            max_decoder_steps: Optional[int] = None) -> Tuple[Tensor, Tensor]:
    embedding = self.embedding
    embedded_inputs = torch.transpose((embedding).forward(inputs, ), 1, 2)
    encoder = self.encoder
    encoder_outputs = (encoder).forward(embedded_inputs, )
    decoder = self.decoder
    _0 = (decoder).forward(encoder_outputs, max_decoder_steps, )
    mel_outputs, gate_outputs, alignments, mel_lengths, = _0
    postnet = self.postnet
    mel_outputs_postnet = (postnet).forward(mel_outputs, )
    mel_outputs_postnet0 = torch.add(mel_outputs, mel_outputs_postnet)
    if torch.gt(torch.size(inputs, 0), 1):
        mel_outputs_postnet2 = (self).parse_output(mel_outputs_postnet0, mel_lengths, )
        mel_outputs_postnet1 = mel_outputs_postnet2
    else:
        mel_outputs_postnet1 = mel_outputs_postnet0
    return (mel_outputs_postnet1, mel_lengths)  #%%

In [96]:

example_text = 'В недрах тундры выдры в гетрах тырят в вёдра ядра кедров. '
cleaned_example_text = "".join([c if c in symbols else " " for c in example_text])

speaker_id = torch.tensor(0).to(device)
embeder = model.tacotron.embedding.to(device)
encoder = model.tacotron.encoder.to(device)
decoder = model.tacotron.decoder.to(device)

input_tensor = torch.LongTensor([symbols.index(c) for c in cleaned_example_text]).unsqueeze(0).to(device)
embedded_inputs = embeder(input_tensor)
embedded_inputs = torch.transpose(embedded_inputs, 1, 2)
encoder_output = encoder(embedded_inputs)
mel_outputs, gate_outputs, alignments, mel_lengths = decoder(encoder_output)

C:\Users\user\.virtualenvs\xlab-ml-service\Lib\site-packages\torch\nn\modules\module.py:1747: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\RNN.cpp:1410.)
  return forward_call(*args, **kwargs)


In [97]:
print("mel_outputs", mel_outputs.shape)  # <- use this as speech data representation
print("gate_outputs", gate_outputs.shape)
print("alignments", alignments.shape)
print("mel_lengths", mel_lengths.shape)

mel_outputs torch.Size([1, 128, 1000])
gate_outputs torch.Size([1, 1000])
alignments torch.Size([1, 1000, 58])
mel_lengths torch.Size([1])


In the context of Silero's Text-to-Speech (TTS) models, particularly those inspired by architectures like Tacotron 2, several key outputs are generated during thesynthesis process:

### Mel Outputs (mel_outputs):

Definition: These are sequences of mel-spectrogram frames representing the synthesized speech's frequency content over time.
Purpose: Mel-spectrograms serve as intermediate representations that are subsequently converted into audible waveforms using vocoders.

### Gate Outputs (gate_outputs):

Definition: These are scalar values produced at each decoder time step, indicating the probability of the synthesis process being complete.
Purpose: Gate outputs help the model determine when to stop generating frames, ensuring efficient and accurate speech synthesis.

### Alignments (alignments):

Definition: Alignments are attention maps that depict the relationship between input text tokens and the generated mel-spectrogram frames.
Purpose: They provide insights into how the model aligns input text with the corresponding speech, which is crucial for understanding and debugging the synthesis process.
### Mel Lengths (mel_lengths):

Definition: These indicate the number of mel-spectrogram frames generated for each input sequence.
Purpose: Mel lengths are essential for batching and padding operations, ensuring that sequences of varying lengths are handled appropriately during training and inference.